In [20]:
# !pip install -q llama-index
# !pip install llama-index
# %pip install llama-index-llms-langchain
# !pip install llama-index-embeddings-langchain
# !pip install llama_index.llms.fireworks

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [21]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.embeddings.cloudflare_workersai import (
    CloudflareWorkersAIEmbeddings,
)

load_dotenv(override=True)
DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')

Settings.llm = ChatOpenAI(
    openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1',
    openai_api_key=DASHSCOPE_API_KEY,
    model_name="qwen2-1.5b-instruct",
    temperature=0.7,
    streaming=True,
)

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')

Settings.embed_model = CloudflareWorkersAIEmbeddings(
    account_id=account_id,
    api_token=api_token,
    model_name="@cf/baai/bge-small-en-v1.5",
)

In [27]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [22]:
documents = SimpleDirectoryReader("../langchain/file").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("Some question about the data should go here")
print(response)

/Users/pangmengting/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  **kwargs: The keyword arguments to the function.


What is the purpose of the trip?


In [28]:
response = query_engine.query("具体行程的第一天行程有哪些？")
print(response)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.cloudflare.com:443
Starting new HTTPS connection (1): api.cloudflare.com:443
DEBUG:urllib3.connectionpool:https://api.cloudflare.com:443 "POST /client/v4/accounts/8483c3ec7a0cbc54a8d660b5b9002b04/ai/run/@cf/baai/bge-small-en-v1.5 HTTP/1.1" 200 None
https://api.cloudflare.com:443 "POST /client/v4/accounts/8483c3ec7a0cbc54a8d660b5b9002b04/ai/run/@cf/baai/bge-small-en-v1.5 HTTP/1.1" 200 None
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node fc4cd8dd-456f-4109-aa11-65a0cc87b4fa] [Similarity score:             0.606345] 第7天 阿尔山国家森林公园（乌苏浪子湖晨雾，驼峰岭天池，大峡谷，杜鹃湖）阿尔山市

摄点： 阿尔山全天摄影创作：乌苏浪子湖晨雾、驼峰岭天池秋色、阿尔山大峡谷、杜鹃湖

- 全天
  早起前往乌苏...
> [Node 07d9a12a-48bf-4987-b881-07cef194649c] [Similarity score:             0.605393] 第1天 各地海拉尔市 ★集合日

- 全天
  全天为活动全天集合日，各地游侠自行前往呼伦贝尔市的首府海拉尔集合。早到的游侠，可以去逛逛呼伦贝尔民族博物馆、成吉思汗广场。初到草原，一定要美美的吃...
> Top 2 nodes:
> [Node fc4cd8dd-456f-4109-aa11-65a0cc87b4fa] [Similarity score:             0.606345] 第7天 阿尔山国家森

In [29]:
response = query_engine.query("简单说一下行程概要？")
print(response)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.cloudflare.com:443
Starting new HTTPS connection (1): api.cloudflare.com:443
DEBUG:urllib3.connectionpool:https://api.cloudflare.com:443 "POST /client/v4/accounts/8483c3ec7a0cbc54a8d660b5b9002b04/ai/run/@cf/baai/bge-small-en-v1.5 HTTP/1.1" 200 None
https://api.cloudflare.com:443 "POST /client/v4/accounts/8483c3ec7a0cbc54a8d660b5b9002b04/ai/run/@cf/baai/bge-small-en-v1.5 HTTP/1.1" 200 None
DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node fc4cd8dd-456f-4109-aa11-65a0cc87b4fa] [Similarity score:             0.609956] 第7天 阿尔山国家森林公园（乌苏浪子湖晨雾，驼峰岭天池，大峡谷，杜鹃湖）阿尔山市

摄点： 阿尔山全天摄影创作：乌苏浪子湖晨雾、驼峰岭天池秋色、阿尔山大峡谷、杜鹃湖

- 全天
  早起前往乌苏...
> [Node 2f4aeba0-3b15-48fc-893a-fc93d4f01662] [Similarity score:             0.608349] 第4天 莫尔道嘎镇白鹿岛白桦林驯鹿黑山头湿地

摄点： 白鹿岛日出晨雾，白桦林驯鹿，草原跑马，湿地日落，牧归场景拍摄

- 早晨
  早起乘坐小车前往【白鹿岛】拍摄大兴安岭河谷的日出晨雾，激流河...
> Top 2 nodes:
> [Node fc4cd8dd-456f-4109-aa11-65a0cc87b4fa] [Similarity score:             0.609956] 第7天 阿尔山国家森

In [0]:
!mkdir data
!wget https://raw.githubusercontent.com/jerryjliu/llama_index/main/examples/paul_graham_essay/data/paul_graham_essay.txt -O data/paul_graham_essay.txt

import os

os.environ['OPENAI_API_KEY'] = 'your valid openai api key'

documents = SimpleDirectoryReader('data').load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")
print(response)

response = query_engine.query("Introduce me Paul Graham")
print(response)
     